Author: Rohan Krishna Balaji  <br>
Course : Simulation Science <br>
Date  : 25.06.2021  <br>
Project : ICTM Analysis, Master's Thesis at Fraunhofer IPT <br>
Email :  rohan.balaji@rwth-aachen.de <br>
Verion : 1.01 <br>

# Introduction
To obtain the complete data set which includes feature from both planning data used in CAM planning and machining data obtained from Parquet fles First is to convert the machining data in machine coordinate system to tool tip data in work space coordinate system through 'Forward Kinematic Transformations' on compensated data. Then each tool tip data is clusterd to the correspoding acqired data points. Then the obtained clasters are averged to generate combined dataset with both planning and machining data.
The dataset is filtered to remove the outliers

In [1]:
# Importing necessary libraries and general settings
import transformation
import compensation
import data_imports
import pandas as pd
import os
import glob
import math
import re
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation as animation
from mpl_toolkits import mplot3d
import numpy as np
from scipy.signal import find_peaks

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
np.set_printoptions(threshold=np.inf)

In [2]:
# Plotting the result obtained during the study
def subplotter(low,high,X,Y,Z,X_inv,Y_inv,Z_inv):
    
    range = np.arange(low,high)
    fig,axs = plt.subplots(3,figsize=(10,15))

    axs[0].plot(range,X[range],label = 'X old', color='blue')
    axs[0].plot(range,X_inv[range],label = 'X new', color='red')
    axs[0].legend() 
    axs[0].set_xlabel('row index (dimension less)')
    axs[0].set_ylabel('X in mm')
    #axs[0].set_title('X data comparison')   
    
    axs[1].plot(range,Y[range], label = 'Y old', color='blue')
    axs[1].plot(range,Y_inv[range], label = 'Y new', color='red') 
    axs[1].legend()
    axs[1].set_xlabel('row index (dimension less)')
    axs[1].set_ylabel('Y in mm')
   # axs[1].set_title('Y data comparison')   
    

    axs[2].plot(range,Z[range], label = 'Z old', color='blue')
    axs[2].plot(range,Z_inv[range] , label = 'Z new', color='red')
    axs[2].legend()
    axs[2].set_xlabel('row index (dimension less)')
    axs[2].set_ylabel('Z in mm')
    #axs[2].set_title('Z data comparison')   
    
    
    fig.canvas.draw()
    fig.canvas.flush_events()


In [3]:
import imageio
import os

def subplotter_2(x1,y1,z1,a1,c1,x2,y2,z2,a2,c2,i,block,angle):
 
    filename = f'{i}.png'
    
    fig,axs = plt.subplots(5,2,figsize=(10,15))

    axs[0][0].plot(x1,label = 'x acquired old'+'--'+str(block)+'--'+str(angle), color='blue')
    axs[0][0].legend() 
    axs[0][0].set_xlabel('row index (dimension less)')
    axs[0][0].set_ylabel('x in mm')   
    
    axs[1][0].plot(y1, label = 'y acquired old'+'--'+str(block)+'--'+str(angle), color='blue')
    axs[1][0].legend()
    axs[1][0].set_xlabel('row index (dimension less)')
    axs[1][0].set_ylabel('y in mm')  

    axs[2][0].plot(z1, label = 'z acquired old'+'--'+str(block)+'--'+str(angle), color='blue')
    axs[2][0].legend()
    axs[2][0].set_xlabel('row index (dimension less)')
    axs[2][0].set_ylabel('z in mm')
    
    axs[3][0].plot(a1, label = 'a acquired old'+'--'+str(block)+'--'+str(angle), color='blue')
    axs[3][0].legend()
    axs[3][0].set_xlabel('row index (dimension less)')
    axs[3][0].set_ylabel('a in mm')
    
    axs[4][0].plot(c1, label = 'c acquired old'+'--'+str(block)+'--'+str(angle), color='blue')
    axs[4][0].legend()
    axs[4][0].set_xlabel('row index (dimension less)')
    axs[4][0].set_ylabel('c in mm')
    
    axs[0][1].plot(x2,label = 'x acquired new'+'--'+str(block)+'--'+str(angle), color='red')
    axs[0][1].legend() 
    axs[0][1].set_xlabel('row index (dimension less)')
    axs[0][1].set_ylabel('x in mm')   
    
    axs[1][1].plot(y2, label = 'y acquired new'+'--'+str(block)+'--'+str(angle), color='red')
    axs[1][1].legend()
    axs[1][1].set_xlabel('row index (dimension less)')
    axs[1][1].set_ylabel('y in mm')  

    axs[2][1].plot(z2, label = 'z acquired new'+'--'+str(block)+'--'+str(angle), color='red')
    axs[2][1].legend()
    axs[2][1].set_xlabel('row index (dimension less)')
    axs[2][1].set_ylabel('z in mm')
    
    axs[3][1].plot(a2, label = 'a acquired new'+'--'+str(block)+'--'+str(angle), color='red')
    axs[3][1].legend()
    axs[3][1].set_xlabel('row index (dimension less)')
    axs[3][1].set_ylabel('a in mm')
    
    axs[4][1].plot(c2, label = 'c acquired new'+'--'+str(block)+'--'+str(angle), color='red')
    axs[4][1].legend()
    axs[4][1].set_xlabel('row index (dimension less)')
    axs[4][1].set_ylabel('c in mm')
    
    plt.savefig(filename)
    plt.close()
    
    return filename

def subplotter_3(x1,y1,z1,x2,y2,z2,i,block):
 
    filename2 = f'{i}.png'
    
    fig,axs = plt.subplots(3,2,figsize=(10,15))

    axs[0][0].plot(x1,label = 'x planning old'+'--'+str(block), color='blue')
    axs[0][0].legend() 
    axs[0][0].set_xlabel('row index (dimension less)')
    axs[0][0].set_ylabel('x in mm')   
    
    axs[1][0].plot(y1, label = 'y planning old'+'--'+str(block), color='blue')
    axs[1][0].legend()
    axs[1][0].set_xlabel('row index (dimension less)')
    axs[1][0].set_ylabel('y in mm')  

    axs[2][0].plot(z1, label = 'z planning old'+'--'+str(block), color='blue')
    axs[2][0].legend()
    axs[2][0].set_xlabel('row index (dimension less)')
    axs[2][0].set_ylabel('z in mm')
    
    axs[0][1].plot(x2,label = 'x planning new'+'--'+str(block), color='red')
    axs[0][1].legend() 
    axs[0][1].set_xlabel('row index (dimension less)')
    axs[0][1].set_ylabel('x in mm')   
    
    axs[1][1].plot(y2, label = 'y planning new'+'--'+str(block), color='red')
    axs[1][1].legend()
    axs[1][1].set_xlabel('row index (dimension less)')
    axs[1][1].set_ylabel('y in mm')  

    axs[2][1].plot(z2, label = 'z planning new'+'--'+str(block), color='red')
    axs[2][1].legend()
    axs[2][1].set_xlabel('row index (dimension less)')
    axs[2][1].set_ylabel('z in mm')

    plt.savefig(filename2)
    plt.close()
    
    return filename2
    
    

In [4]:
def gif_gen(filenames,gifname):    
    frame = []
    for filename in filenames:
        image = imageio.imread(filename)
        frame.append(image)
        
    imageio.mimsave(gifname+'.gif', frame, format='GIF', duration=1)
        
    # Remove files
    for filename in set(filenames):
        os.remove(filename)

In [16]:
#blocks = [1030, 2030, 3030, 4030, 5030, 6030, 7030]
#angles = list(np.linspace(0,348,30))
#dir_final_save  = 'D:/rohan/thesis/Projects/Machine_Learning/Data/Combined_Data/'

#blocks = [1030, 2030, 3030, 4030, 6030, 7030]
blocks = [1030]
angles = [12,72,84]
dir_final_save1  = 'D:/rohan/thesis/Projects/Machine_Learning/Data/Manufacturing_Data/'
dir_final_save2  = 'D:/rohan/thesis/Projects/Machine_Learning/Data_2021/Parquet/'

i = 0 

filenames = []
for block in blocks:
    for angle in angles:
        i = i+1
        print(block, angle)
        
        #df_m = pd.read_parquet('ProgNumber-'+str(block)+'--Blade--'+str(int(angle))+'_DownsampledData'+'.parquet')
        df_m1 = pd.read_parquet(str(dir_final_save1)+'ProgNumber-'+str(block)+'--Blade--'+str(angle)+'_DownsampledData'+'.parquet')
        
        x1 = df_m1['MachineX'].copy(deep=True).to_numpy()
        y1 = df_m1['MachineY'].copy(deep=True).to_numpy()
        z1 = df_m1['MachineZ'].copy(deep=True).to_numpy()
        a1 = np.radians(df_m1['MachineA']).copy(deep=True).to_numpy()
        c1 = np.radians(df_m1['MachineC']).copy(deep=True).to_numpy()
        
        df_m2 = pd.read_parquet(str(dir_final_save2)+str(int(angle))+'-'+str(block+3)+'-0'+'.parquet')
        df_m2 = df_m2.rename(columns=df_m2.iloc[0])
        df_m2 = df_m2.iloc[1: , :]
        
        x2 = pd.to_numeric(df_m2['DynamicData     McX']).to_numpy()
        y2 = pd.to_numeric(df_m2['DynamicData     McY']).to_numpy()
        z2 = pd.to_numeric(df_m2['DynamicData     McZ']).to_numpy()
        a2 = pd.to_numeric(df_m2['DynamicData     Mc4th']).to_numpy()
        c2 = pd.to_numeric(df_m2['DynamicData     Mc5th']).to_numpy()
        
        #x2 = pd.to_numeric(df_m2['DynamicData     WcX']).to_numpy()
        #y2 = pd.to_numeric(df_m2['DynamicData     WcY']).to_numpy()
        #z2 = pd.to_numeric(df_m2['DynamicData     WcZ']).to_numpy()
        #a2 = pd.to_numeric(df_m2['DynamicData     Wc4th']).to_numpy()
        #c2 = pd.to_numeric(df_m2['DynamicData     Wc5th']).to_numpy()
        
        
        filenames.append(subplotter_2(x1,y1,z1,a1,c1,x2,y2,z2,a2,c2,i,block,angle))
        

gif_gen(filenames,'combined_acquired_mcx_1030...')

1030 12
1030 72
1030 84
2030 12
2030 72
2030 84
3030 12
3030 72
3030 84
4030 12
4030 72
4030 84
6030 12
6030 72
6030 84
7030 12
7030 72
7030 84


In [31]:
#blocks = [1030, 2030, 3030, 4030, 5030, 6030, 7030]
#angles = list(np.linspace(0,348,30))
#dir_final_save  = 'D:/rohan/thesis/Projects/Machine_Learning/Data/Combined_Data/'

#blocks = [1030, 2030, 3030, 4030, 6030, 7030]
blocks = [7030]
angles = [12,72,84]
dir_final_save1  = 'D:/rohan/thesis/Projects/Machine_Learning/Data/Planning_Data/'
dir_final_save2  = 'D:/rohan/thesis/Projects/Machine_Learning/Data_2021/Toolpath/'

i = 0 

filenames2 = []
for block in blocks:
    i = i+1
    print(block)
        
    #df_m = pd.read_parquet('ProgNumber-'+str(block)+'--Blade--'+str(int(angle))+'_DownsampledData'+'.parquet')
    df_m1 = pd.read_excel(dir_final_save1+'Planning_Data_OP'+str(block)+'.xlsx', engine='openpyxl')
    
    x1 = df_m1['Tool Tip Point X'].copy(deep=True).to_numpy()
    y1 = df_m1['Tool Tip Point Y'].copy(deep=True).to_numpy()
    z1 = df_m1['Tool Tip Point Z'].copy(deep=True).to_numpy()
        
    df_m2 = pd.read_csv(dir_final_save2+'OP'+str(block)+'_TD_and_WD'+'.csv',delimiter= ';')
        
    x2 = pd.to_numeric(df_m2['X_TTP Q [mm]']).to_numpy()
    y2 = pd.to_numeric(df_m2['Y_TTP  [mm]']).to_numpy()
    z2 = pd.to_numeric(df_m2['Z_TTP  [mm]']).to_numpy()
            
    filenames2.append(subplotter_3(x1,y1,z1,x2,y2,z2,i,block))
       
gif_gen(filenames2,'combined_planning_7030')

7030


In [32]:
block = 7030

dir_final_save1  = 'D:/rohan/thesis/Projects/Machine_Learning/Data/Planning_Data/'
dir_final_save2  = 'D:/rohan/thesis/Projects/Machine_Learning/Data_2021/Toolpath/'

df_m1 = pd.read_excel(dir_final_save1+'Planning_Data_OP'+str(block)+'.xlsx', engine='openpyxl')
    
x1 = pd.to_numeric(df_m2['DynamicData     WcX']).to_numpy()
y1 = pd.to_numeric(df_m2['DynamicData     WcY']).to_numpy()
z1 = pd.to_numeric(df_m2['DynamicData     WcZ']).to_numpy()
        
df_m2 = pd.read_csv(dir_final_save2+'OP'+str(block)+'_TD_and_WD'+'.csv',delimiter= ';')
        
x2 = pd.to_numeric(df_m2['X_TTP Q [mm]']).to_numpy()
y2 = pd.to_numeric(df_m2['Y_TTP  [mm]']).to_numpy()
z2 = pd.to_numeric(df_m2['Z_TTP  [mm]']).to_numpy()

subplotter(0,20000,x1,y2,z1,x2,y2,z2)

KeyError: 'DynamicData     WcX'

Path locations (folder location) for respective files

# New data

In [ ]:
df_m = pd.read_parquet('D:/rohan/thesis/Projects/Machine_Learning/Data_2021/Parquet/96-1030-0.parquet')
angle = 96
df_p = pd.read_csv('D:/rohan/thesis/Projects/Machine_Learning/Data_2021/Toolpath/OP1030.csv',delimiter= ';')

df_m = df_m.rename(columns=df_m.iloc[0])
df_m = df_m.iloc[1: , :]
print(df_m.columns)

x = pd.to_numeric(df_m['DynamicData     McX']).to_numpy()
y = pd.to_numeric(df_m['DynamicData     McY']).to_numpy()
z = pd.to_numeric(df_m['DynamicData     McZ']).to_numpy()
a = pd.to_numeric(df_m['DynamicData     Mc4th']).to_numpy()
c = pd.to_numeric(df_m['DynamicData     Mc5th']).to_numpy()

tool_tip_X = df_p['X_TTP Q [mm]'].to_numpy()
tool_tip_Y = df_p['Y_TTP  [mm]'].to_numpy()
tool_tip_Z = df_p['Z_TTP  [mm]'].to_numpy()
X_inv=tool_tip_X
Y_inv=tool_tip_Y
Z_inv=tool_tip_Z

In [ ]:
df_m.head()

# Old Data

In [ ]:
df_m = pd.read_parquet('D:/rohan/thesis/Projects/Machine_Learning/Data/Manufacturing_Data/ProgNumber-1030--Blade--180_DownsampledData.parquet')
angle = 180
df_p = pd.read_excel('D:/rohan/thesis/Projects/Machine_Learning/Data/Planning_Data/Planning_Data_OP1030.xlsx', engine='openpyxl') 

x = df_m['MachineX'].copy(deep=True).to_numpy()
y = df_m['MachineY'].copy(deep=True).to_numpy()
z = df_m['MachineZ'].copy(deep=True).to_numpy()
a = np.radians(df_m['MachineA']).copy(deep=True).to_numpy()
c = np.radians(df_m['MachineC']).copy(deep=True).to_numpy()

tool_tip_X = df_p['Tool Tip Point X'].to_numpy()
tool_tip_Y = df_p['Tool Tip Point Y'].to_numpy()
tool_tip_Z = df_p['Tool Tip Point Z'].to_numpy()
X_inv=tool_tip_X
Y_inv=tool_tip_Y
Z_inv=tool_tip_Z


In [ ]:
df_m.head()

In [ ]:
plt.plot(z)

### Forward Transformation:
Forward transformation is performed by applying rotational and transitional
transformations on the machine points and orientation in the machine coordi-
nate system.
There are two ways to interpret the transformations. Firstly, by pre-multiplication
of all the transformation matrices with respect to base coordinate system (i.e
Machine Coordinate System). Second method is the apply post- multiplica-
tion this can be understood as applying 'Relative Transformation'.
The notation used for machine data points obtained by paraqet fles are
x,y,z,a,c in machine coordinate system. The tool tip and orientations obtained
in workpiece coordinate system X,Y,Z,I,J,K. This is implemented in transformation.py script.

<img src="images/transformation.png">

In [ ]:
import numpy as np
from scipy.optimize import least_squares
from scipy.optimize import root

size1 = x.shape[0]
#angle = 96

OprOm_x = 0
OprOm_y = 0
OprOm_z = -510.483#+167.85

OtOa_x = 0.0011
OtOa_y =-0.0093  
OtOa_z =-600.2241

OtOc_x = 0.0304
OtOc_y = -0.0028
OtOc_z = -79.7629

l = 0
tl = 226.6649

IT_x=0
IT_y=0
IT_z=1

T_x = 0
T_y = 0
T_z = tl-l
T_CentertoTip = l

OprOm_x = 0
OprOm_y = 0
OprOm_z = -510.4830#+167.85

# Zeropoint to A Table
l = 0
tl = 226.6649
OtOa_x =0.008 #19700
OtOa_y =-0.0004 #19701
OtOa_z =-600.2464 #19702
# C excentration from Table
OtOc_x = -0.0185 #19703
OtOc_y = -0.0035 #19704
OtOc_z = -79.7629 #19705
# Tool dimensions
T_x = 0
T_y = 0
T_z = tl#-l)
T_CentertoTip = l
    

size = size1
ones = np.ones((size,))
zeros = np.zeros((size,))

init_C = np.array([[np.cos(np.radians(-angle))*ones,  -np.sin(np.radians(-angle))*ones, zeros, zeros],
                    [np.sin(np.radians(-angle))*ones,  np.cos(np.radians(-angle))*ones, zeros, zeros],
                    [zeros,                            zeros,                           ones,  zeros],
                    [zeros,                             zeros,                          zeros, ones]])

        
matrix_translationA = np.array([[ones,  zeros, zeros,  np.ones((size,))*(-OtOa_x-T_x)],
                                [zeros, ones,  zeros,  np.ones((size,))*(-OtOa_y-T_y)],
                                [zeros, zeros, ones,   np.ones((size,))*(-OtOa_z-T_z)],
                                [zeros, zeros, zeros,  ones  ]])
        
matrix_rotationA = np.array([[ones,   zeros,       zeros,      zeros],
                            [zeros,   np.cos(a),  -np.sin(a),  zeros],
                            [zeros,   np.sin(a),   np.cos(a),  zeros],
                            [zeros,   zeros,       zeros       ,ones ]],)
         
matrix_translationC = np.array([[ones,  zeros, zeros,  -np.ones((size,))*OtOc_x],
                                [zeros, ones,  zeros,  -np.ones((size,))*OtOc_y],
                                [zeros, zeros, ones,   -np.ones((size,))*OtOc_z],
                                [zeros, zeros, zeros,   ones  ]])

matrix_rotationC = np.array([[np.cos(c), -np.sin(c), zeros, zeros],
                             [np.sin(c),   np.cos(c), zeros, zeros],
                             [zeros,       zeros,     ones,  zeros],
                             [zeros,       zeros,     zeros, ones]])
        
matrix_back_translation = np.array([[ones,  zeros, zeros,   np.ones((size,))*(-OprOm_x +OtOa_x + OtOc_x)  ],
                                    [zeros, ones,  zeros,   np.ones((size,))*(-OprOm_y + OtOa_y + OtOc_y) ],
                                    [zeros, zeros, ones,    np.ones((size,))*(-OprOm_z + OtOa_z + OtOc_z) ],
                                    [zeros, zeros, zeros,   ones                                         ]])
        

initial_tool_position = np.array([[ones*[IT_x]],
                                  [ones*[IT_y]],
                                  [ones*[IT_z]],
                                  [ones*[0]]])
      
machine_points_xyz   = np.array([[x],
                                 [y],
                                 [z],
                                 [ones]])
        


        #Forward transformation <-- Back Translation * Rotation on C axis * Translation in C axis * Rotation on A axis * Translation in A

forward_transformation = np.transpose(init_C, (2,0,1)) @ \
                                      np.transpose(matrix_back_translation,(2,0,1))@ \
                                      np.transpose(matrix_rotationC, (2,0,1)) @ \
                                      np.transpose(matrix_translationC, (2,0,1))@ \
                                      np.transpose(matrix_rotationA, (2,0,1))@ \
                                      np.transpose(matrix_translationA, (2,0,1))
        
tool_position_workpiece_CS = forward_transformation @ np.transpose(machine_points_xyz,(2,0,1))
tool_position_workpiece_CS = np.transpose(tool_position_workpiece_CS,(1,2,0))
        
tool_orientation_workpiece_CS = forward_transformation @ np.transpose(initial_tool_position,(2,0,1))
tool_orientation_workpiece_CS = np.transpose(tool_orientation_workpiece_CS,(1,2,0))

X = tool_position_workpiece_CS[0,0,:]
Y = tool_position_workpiece_CS[1,0,:]
Z = tool_position_workpiece_CS[2,0,:]

In [ ]:
subplotter(0,10000,X,Y,Z,X_inv,Y_inv,Z_inv)

In [ ]:
X = pd.to_numeric(df_m['DynamicData     WcX']).to_numpy()
Y = pd.to_numeric(df_m['DynamicData     WcY']).to_numpy()
Z = pd.to_numeric(df_m['DynamicData     WcZ']).to_numpy()

In [ ]:
X = pd.to_numeric(df_m['DynamicData     WcX']).to_numpy()
Y = pd.to_numeric(df_m['DynamicData     WcY']).to_numpy()
Z = pd.to_numeric(df_m['DynamicData     WcZ']).to_numpy()
angle = 96
# 48 , 32.5         #0 , 15.5
size = X.shape[0]
ones = np.ones((size,))
zeros = np.zeros((size,))
init_C = np.array([[np.cos(np.radians(-angle))*ones,  -np.sin(np.radians(-angle))*ones, zeros, zeros],
                    [np.sin(np.radians(-angle))*ones,  np.cos(np.radians(-angle))*ones, zeros, zeros],
                    [zeros,                            zeros,                           ones,  zeros],
                    [zeros,                             zeros,                          zeros, ones]])

machine_points_xyz   = np.array([[X],
                                 [Y],
                                 [Z],
                                 [ones]])
        


        #Forward transformation <-- Back Translation * Rotation on C axis * Translation in C axis * Rotation on A axis * Translation in A

forward_transformation = np.transpose(init_C, (2,0,1))
        
tool_position_workpiece_CS = forward_transformation @ np.transpose(machine_points_xyz,(2,0,1))
tool_position_workpiece_CS = np.transpose(tool_position_workpiece_CS,(1,2,0))

In [ ]:
subplotter(0,df_m.shape[0],X,Y,Z,X_inv,Y_inv,Z_inv)

In [ ]:
#subplotter(1000,28900,X,Y,Z,X_inv,Y_inv,Z_inv)

In [ ]:
subplotter(20000,21000,X,Y,Z,X_inv,Y_inv,Z_inv)

In [ ]:
"""
range1 = np.arange(1000,34900)
X = X[range1]
Y = Y[range1]
Z = Z[range1]

tool_tip_X = tool_tip_X[range1]
tool_tip_Y = tool_tip_Y[range1]
tool_tip_Z = tool_tip_Z[range1]
 
X_inv = tool_tip_X
Y_inv = tool_tip_Y
Z_inv = tool_tip_Z
"""

### Finding the closest points
Here the distances are calculated, for each aquired data point, forward transformation was performed above, and now for each of those poits the disace for all planning points are calculated. Partition the data into 5 chuncks and find the closest distance points in those chucks, if we use full set at once, we may match up with far way points, which were causing lots of outliers.the position closest the iterator is considered, since the far away values are thus avoided from matching. store both position and the value in two separate lists

In [ ]:
# Here the distances are calculated, for each aquired data point, forward transformation was performed above,
# and now for each of those poits the disace for all planning points are calculated.
lst1 = []
lst2 = []

dist  = np.zeros(len(tool_tip_X))
print('Enter the tolerance number value, example 3 mm')
#tol = input()
tol = 3
count = 0 
k = 5

for i in np.arange(len(X)):   # 47917
    
    dist = (((tool_tip_X-X[i])**2+(tool_tip_Y-Y[i])**2+(tool_tip_Z-Z[i])**2)**(1/2))
    
    # partition the data into 5 chuncks and find the closest distance points in those chucks, if we use full set at once, 
    # we may match up with far way points, which were causing lots of outliers
    
    pos = np.argpartition(dist, k)
    pos = pos[:k]
    min_val = dist[pos]
    #print(pos,min_val)
    
    # the position closest the iterator is considered, since the far away values are thus avoided from matching
    pos = pos[np.argmin(abs(pos - i))]
    min_val = dist[pos]
    
    #print(i,pos,min_val)
    # store both position and the value in two separate lists
    if(abs(min_val<=float(tol))): 
        lst1.append(pos)
        lst2.append(min_val)
        count = count+1
        print(i,pos,min_val,count)
    else:
        lst1.append(np.nan)
        lst2.append(np.nan)
   

In [ ]:
print(np.shape(lst1))

In [ ]:
# Seeing all the unique clusers and number of points in it (Just for understanding)
(unique,count) = np.unique(lst1, return_counts=True)

df_freq = pd.DataFrame({'Unique Cluster Points':np.array(unique),'Count':np.array(count)})
df_freq = df_freq.dropna()
df_freq.iloc[:]

In [ ]:

df_cluster = pd.DataFrame({'cluster index in planning data':np.array(lst1),'distance error':np.array(lst2)})
#df_cluster = df_cluster.dropna()
df_cluster.iloc[22000:22100]

### Clustering:
For a given planning point, there are none or multiple matching acquried points, those are found in this section. For each planning datapoint, see whih index match with the stored index for minimum distance. Those can be considered as the cluster points and the corresponding distance. 

<img src="images/cluster.png">

In [ ]:
# Clustering: for a given planning point, there are none or multiple matching acquried points, those are found in this section of code 
lst3 = []
tcp_val = []
# For each planning datapoint, see whih index match with the stored index for minimum distance.
# Those can be considered as the cluster points and the corresponding distance
for i in np.arange(len(tool_tip_Y)):     # each planning point
    
    matching = np.where(np.array(lst1)==i)    # see what all machine points matches
    tcp_val.append([lst2[index] for index in matching[0]])          # get the same minmum tcp error distance 
    lst3.append(matching[0])
    print(i,matching[0])
    

### Visualization

In [ ]:
# list for plotting
pltlst = []
for i in np.arange(len(tool_tip_X)):
    if (len(lst3[i]) != 0):
        pltlst.append(i)
        #print(i,lst3[i])

In [ ]:
# Plot to show clusters
xdata, ydata, zdata = np.zeros(len(pltlst),),np.zeros(len(pltlst),),np.zeros(len(pltlst),)

plt.figure(figsize=(10,15))
ax = plt.axes(projection='3d')
ax.set_zlim3d([0,30])
ax.set_xlabel('$X$ in mm', fontsize=20)
ax.set_ylabel('$Y$ in mm', fontsize=20)
ax.set_zlabel('$Z$ in mm', fontsize=20)

for i in np.arange(len(pltlst)):
    xdata[i] = tool_tip_X[pltlst[i]]
    ydata[i] = tool_tip_Y[pltlst[i]]
    zdata[i] = tool_tip_Z[pltlst[i]]

ax.scatter3D(xdata, ydata, zdata, color = 'red',label = 'Planning Data')
ax.legend()

xdata2, ydata2, zdata2 = np.zeros(len(pltlst),),np.zeros(len(pltlst),),np.zeros(len(pltlst),)

for i in np.arange(len(pltlst)):
    xdata2[i] = np.average(X[tuple([lst3[pltlst[i]]])])
    ydata2[i] = np.average(Y[tuple([lst3[pltlst[i]]])])
    zdata2[i] = np.average(Z[tuple([lst3[pltlst[i]]])])

ax.scatter3D(xdata2, ydata2, zdata2, color = 'blue',label = 'Machine Data Avg')
ax.legend()

In [ ]:
#subplotter(0,xdata.shape[0],xdata2,ydata2,zdata2,xdata,ydata,zdata)
subplotter(0,19000,xdata2,ydata2,zdata2,xdata,ydata,zdata)

In [ ]:
num = -10
xdata, ydata, zdata = np.zeros(len(lst3[pltlst[num]]),),np.zeros(len(lst3[pltlst[num]]),),np.zeros(len(lst3[pltlst[num]]),)

plt.figure(figsize=(6,6))
ax = plt.axes(projection='3d')

ax.scatter3D(tool_tip_X[pltlst[num]],tool_tip_Y[pltlst[num]] , tool_tip_Z[pltlst[num]],label='Plannig Data Point')

for i in np.arange(len(lst3[pltlst[num]])):
    xdata[i] = X[lst3[pltlst[num]]][i]
    ydata[i] = Y[lst3[pltlst[num]]][i]
    zdata[i] = Z[lst3[pltlst[num]]][i]
    
ax.scatter3D(xdata, ydata, zdata, label = 'Clustered Acquired Data Points')
ax.set_xlabel('X in mm', fontsize=10)
ax.set_ylabel('Y in mm', fontsize=10)
ax.set_zlabel('Z in mm', fontsize=10)
plt.legend()

In [ ]:
#for num in np.arange(len(pltlst)):
print('Input two numbers between 0 and {} with small difference for plots exaple 10000 then 10010'.format(len(pltlst)))
#lim1 = input()
#lim2 = input()

lim1 = 0
lim2 = 1

for num in np.arange(len(pltlst[int(lim1):int(lim2)])):
    xdata, ydata, zdata = np.zeros(len(lst3[pltlst[num]]),),np.zeros(len(lst3[pltlst[num]]),),np.zeros(len(lst3[pltlst[num]]),)
    
    fig = plt.figure(figsize=(6,6))
    print(num)
    ax = plt.axes(projection='3d')

    ax.scatter3D(tool_tip_X[pltlst[num]],tool_tip_Y[pltlst[num]] , tool_tip_Z[pltlst[num]], label='Plannig Data Point')

    for i in np.arange(len(lst3[pltlst[num]])):
        xdata[i] = X[lst3[pltlst[num]]][i]
        ydata[i] = Y[lst3[pltlst[num]]][i]
        zdata[i] = Z[lst3[pltlst[num]]][i]
    
    ax.scatter3D(xdata, ydata, zdata, label = 'Clustered Acquired Data Points')
    ax.set_xlabel('X in mm', fontsize=10)
    ax.set_ylabel('Y in mm', fontsize=10)
    ax.set_zlabel('Z in mm', fontsize=10)
    plt.legend()
    plt.pause(0.1)
   

In [ ]:
# for the clusterd points the acquired points are averaged
lst4 = []
for i in np.arange(len(lst3)):
    lst4.append(df_m.iloc[list(lst3[i])].mean(axis=0))
    
mean_m = pd.concat(lst4,axis=1).T
mean_m.iloc[-100:-1]

In [ ]:
# distances are averaged
tcp_avg = [(lambda x: sum(x)/len(x))(item) if len(item)!=0 else np.nan for item in tcp_val]
len(tcp_avg)
#tcp_avg

In [ ]:
# Now the final dataframe with plannind data and the corresponding averaged acquired data is obtained 
final_df = pd.concat([df_p,mean_m, pd.DataFrame({'tcp_error':tcp_avg})], axis=1)
final_df = final_df.drop(['Level','Step'],axis=1)
final_df = final_df.dropna()
print(final_df.shape)
final_df.iloc[0:10]

In [ ]:
final_df['block'] = block
final_df['angle'] = angle

In [ ]:
print(final_df.shape)
final_df.iloc[0:10]

In [ ]:
final_df = final_df.iloc[int(final_df.shape[0]*0.05):-int(final_df.shape[0]*0.05),:]

In [ ]:
print(final_df.shape)
final_df.iloc[0:10]

### Using Combined dataset Peaks Elimination:
Inspite of the average, there are some high impuse peaks observed in the data (Run the visualiation.ipnby), these must be removed to obtain a flawless, reliable dataset. Using machine data from newly created combined dataset same forward transformation is performed as discribled above. by observing the data, it can be seen that the using just one of the components (in this case Y) all the outlers can be eliminated. Two types of spikes are typicalled observed (check visualizaion by runnung  vizualization.ipnyb) one in positive direction and othes in negetive direction, so for robostness both are considered. At the same time, it is not wise to remove the original expected trajectory of the tool path in begining and the end, so especially for the positive direction they are intentionally not removed. if each peak on a plateu has to be removed its very expensive, so if two neighbous are consicutively classifies as peaks (given by count), it is assumed as plateu and 100 datapoint sare removed. This is a reasonable compromise in accuracy for significant speedup

<img src="images/peak_eli.png">

### Visulaization of outliers peaks

In [ ]:
x_final = final_df['MachineX'].copy(deep=True)                 # using machine data from newly created combined dataset
y_final = final_df['MachineY'].copy(deep=True)
z_final = final_df['MachineZ'].copy(deep=True)
a_final = final_df['MachineA'].copy(deep=True)
c_final = final_df['MachineC'].copy(deep=True)

size1_final = x_final.shape[0]

# converting pandas series to numpy array
x_final = x_final.to_numpy()
y_final = y_final.to_numpy()
z_final = z_final.to_numpy()
a_final = a_final.to_numpy(dtype =  np.float64)
a_final = np.deg2rad(a_final)
c_final = c_final.to_numpy(dtype =  np.float64)
c_final = np.deg2rad(c_final)

tool_tip_X_final = final_df['Tool Tip Point X'].to_numpy()        # using newly created combined dataset
tool_tip_Y_final = final_df['Tool Tip Point Y'].to_numpy()
tool_tip_Z_final = final_df['Tool Tip Point Z'].to_numpy()
    
X_inv_final = tool_tip_X_final
Y_inv_final = tool_tip_Y_final
Z_inv_final = tool_tip_Z_final
    
#compensation_values
compensation_values = compensation_values_df.to_numpy()

# Within each cube we have ranges defined in x,y,z for the machine position

x_range = np.arange(-200,201,100)
y_range = np.arange(-300,301,150)
z_range = np.arange(-500,1,50)

obj3_final = compensation.Compensation(compensation_values,x_range,y_range,z_range)

# Caclculation of compensation error values based on machine positions obtained through inverse transformation
deltaX1_final, deltaY1_final, deltaZ1_final,deltaI1_final,deltaJ1_final,deltaK1_final = obj3_final.calculate(x_final,y_final,z_final)  
size3_final = x_final.shape[0]
conc3_final = np.concatenate((deltaX1_final.reshape(size3_final,1),deltaY1_final.reshape(size3_final,1),deltaZ1_final.reshape(size3_final,1),deltaI1_final.reshape(size3_final,1),deltaJ1_final.reshape(size3_final,1),deltaK1_final.reshape(size3_final,1)),axis=1)

# prininting the compensation error values
df_obj3_final = pd.DataFrame(conc3_final, columns=['deltaX1 final','deltaY1 final','deltaZ1 final','deltaI1 final','deltaJ1 final','deltaK1 final'])
df_obj3_final.head(5)  

x_compensated_final = x_final + deltaX1_final*10**-3    # since given compensation is to be converted from microns to mm (10**-6 x 10**3 = 10**-3)
y_compensated_final = y_final + deltaY1_final*10**-3
z_compensated_final = z_final + deltaZ1_final*10**-3

conc4_final = np.concatenate((x_compensated_final.reshape(size3_final,1),y_compensated_final.reshape(size3_final,1),z_compensated_final.reshape(size3_final,1)),axis=1)
df_obj4_final = pd.DataFrame(conc4_final, columns=['x_compensated final','y_compensated final','z_compensated final'])

obj_final = transformation.Transformation(size1_final,angle)
# Forward Transformation fuction:
# Input : Machine points in machine coordinate system
# Output: returns too tip points and orientation in workpiece coordinate system
tool_position_workpiece_CS_final, tool_orientation_workpiece_CS_final = obj_final.forward(x_compensated_final,y_compensated_final,z_compensated_final,a_final,c_final)

X_final = tool_position_workpiece_CS_final[0,0,:]
Y_final = tool_position_workpiece_CS_final[1,0,:]
Z_final = tool_position_workpiece_CS_final[2,0,:]

I_final = tool_orientation_workpiece_CS_final[0,0,:]
J_final = tool_orientation_workpiece_CS_final[1,0,:]
K_final = tool_orientation_workpiece_CS_final[2,0,:]

# Verification the correctness of code, i.e on applying reverse transformation on the forward transformation we should get same values
machine_points_xyz_final, machine_direction_ac_final = obj_final.backward(X_final,Y_final,Z_final,I_final,J_final,K_final)

x_out_final = machine_points_xyz_final[0,0,:]
y_out_final = machine_points_xyz_final[1,0,:]
z_out_final = machine_points_xyz_final[2,0,:]

a_out_final = machine_direction_ac_final[0,0,:]
c_out_final = machine_direction_ac_final[1,0,:]

conc1_final = np.concatenate((x_final.reshape(size1_final,1),x_out_final.reshape(size1_final,1),y_final.reshape(size1_final,1),y_out_final.reshape(size1_final,1),z_final.reshape(size1_final,1),z_out_final.reshape(size1_final,1),a_final.reshape(size1_final,1),a_out_final.reshape(size1_final,1),c_final.reshape(size1_final,1),c_out_final.reshape(size1_final,1)),axis=1)
df_obj1_final = pd.DataFrame(conc1_final, columns=['x final','x_out final','y final','y_out final','z final','z_out final','a final','a_out final','c final','c_out'])

conc11_final = np.concatenate((x_final.reshape(size1_final,1),y_final.reshape(size1_final,1),z_final.reshape(size1_final,1),a_final.reshape(size1_final,1),c_final.reshape(size1_final,1),X_final.reshape(size1_final,1),Y_final.reshape(size1_final,1),Z_final.reshape(size1_final,1),I_final.reshape(size1_final,1),J_final.reshape(size1_final,1),K_final.reshape(size1_final,1)),axis=1)
dfout11_final = pd.DataFrame(conc11_final, columns=['x final','y final','z final','a final','c final','X final','Y final','Z final','I final','J final','K final'])
    
subplotter(0,final_df.shape[0],X_final,Y_final,Z_final,X_inv_final,Y_inv_final,Z_inv_final)   # using combined dataset

In [ ]:
plt.figure(figsize=(10,15))
ax1 = plt.axes(projection='3d')
ax1.set_zlim3d([0,30])

ax1.set_xlabel('$X$ in mm', fontsize=20)
ax1.set_ylabel('$Y$ in mm', fontsize=20)
ax1.set_zlabel('$Z$ in mm', fontsize=20)

ax1.scatter3D(X_inv_final, Y_inv_final, Z_inv_final, color = 'red',label = 'Planning Data')
ax1.legend()

ax1.scatter3D(X_final, Y_final, Z_final, color = 'blue',label = 'Machine Data Avg')
ax1.legend()

In [ ]:
def remove_peaks(final_df, compensation_values_df):
    x_final = final_df['MachineX'].copy(deep=True)                 # using machine data from newly created combined dataset
    y_final = final_df['MachineY'].copy(deep=True)
    z_final = final_df['MachineZ'].copy(deep=True)
    a_final = final_df['MachineA'].copy(deep=True)
    c_final = final_df['MachineC'].copy(deep=True)

    size1_final = x_final.shape[0]

    # converting pandas series to numpy array
    x_final = x_final.to_numpy()
    y_final = y_final.to_numpy()
    z_final = z_final.to_numpy()
    a_final = a_final.to_numpy(dtype = np.float64)
    a_final = np.deg2rad(a_final)
    c_final = c_final.to_numpy(dtype = np.float64)
    c_final = np.deg2rad(c_final)

    tool_tip_X_final = final_df['Tool Tip Point X'].to_numpy()        # using newly created combined dataset
    tool_tip_Y_final = final_df['Tool Tip Point Y'].to_numpy()
    tool_tip_Z_final = final_df['Tool Tip Point Z'].to_numpy()
    
    X_inv_final = tool_tip_X_final
    Y_inv_final = tool_tip_Y_final
    Z_inv_final = tool_tip_Z_final
    
    #compensation_values
    compensation_values = compensation_values_df.to_numpy()

    # Within each cube we have ranges defined in x,y,z for the machine position

    x_range = np.arange(-200,201,100)
    y_range = np.arange(-300,301,150)
    z_range = np.arange(-500,1,50)

    obj3_final = compensation.Compensation(compensation_values,x_range,y_range,z_range)

    # Caclculation of compensation error values based on machine positions obtained through inverse transformation
    deltaX1_final, deltaY1_final, deltaZ1_final,deltaI1_final,deltaJ1_final,deltaK1_final = obj3_final.calculate(x_final,y_final,z_final)  
    size3_final = x_final.shape[0]
    conc3_final = np.concatenate((deltaX1_final.reshape(size3_final,1),deltaY1_final.reshape(size3_final,1),deltaZ1_final.reshape(size3_final,1),deltaI1_final.reshape(size3_final,1),deltaJ1_final.reshape(size3_final,1),deltaK1_final.reshape(size3_final,1)),axis=1)

    # prininting the compensation error values
    df_obj3_final = pd.DataFrame(conc3_final, columns=['deltaX1 final','deltaY1 final','deltaZ1 final','deltaI1 final','deltaJ1 final','deltaK1 final'])
    df_obj3_final.head(5)  

    x_compensated_final = x_final + deltaX1_final*10**-3    # since given compensation is to be converted from microns to mm (10**-6 x 10**3 = 10**-3)
    y_compensated_final = y_final + deltaY1_final*10**-3
    z_compensated_final = z_final + deltaZ1_final*10**-3

    conc4_final = np.concatenate((x_compensated_final.reshape(size3_final,1),y_compensated_final.reshape(size3_final,1),z_compensated_final.reshape(size3_final,1)),axis=1)
    df_obj4_final = pd.DataFrame(conc4_final, columns=['x_compensated final','y_compensated final','z_compensated final'])

    obj_final = transformation.Transformation(size1_final,angle)
    # Forward Transformation fuction:
    # Input : Machine points in machine coordinate system
    # Output: returns too tip points and orientation in workpiece coordinate system
    tool_position_workpiece_CS_final, tool_orientation_workpiece_CS_final = obj_final.forward(x_compensated_final,y_compensated_final,z_compensated_final,a_final,c_final)

    X_final = tool_position_workpiece_CS_final[0,0,:]
    Y_final = tool_position_workpiece_CS_final[1,0,:]
    Z_final = tool_position_workpiece_CS_final[2,0,:]

    I_final = tool_orientation_workpiece_CS_final[0,0,:]
    J_final = tool_orientation_workpiece_CS_final[1,0,:]
    K_final = tool_orientation_workpiece_CS_final[2,0,:]

    # Verification the correctness of code, i.e on applying reverse transformation on the forward transformation we should get same values
    machine_points_xyz_final, machine_direction_ac_final = obj_final.backward(X_final,Y_final,Z_final,I_final,J_final,K_final)

    x_out_final = machine_points_xyz_final[0,0,:]
    y_out_final = machine_points_xyz_final[1,0,:]
    z_out_final = machine_points_xyz_final[2,0,:]

    a_out_final = machine_direction_ac_final[0,0,:]
    c_out_final = machine_direction_ac_final[1,0,:]

    conc1_final = np.concatenate((x_final.reshape(size1_final,1),x_out_final.reshape(size1_final,1),y_final.reshape(size1_final,1),y_out_final.reshape(size1_final,1),z_final.reshape(size1_final,1),z_out_final.reshape(size1_final,1),a_final.reshape(size1_final,1),a_out_final.reshape(size1_final,1),c_final.reshape(size1_final,1),c_out_final.reshape(size1_final,1)),axis=1)
    df_obj1_final = pd.DataFrame(conc1_final, columns=['x final','x_out final','y final','y_out final','z final','z_out final','a final','a_out final','c final','c_out'])

    conc11_final = np.concatenate((x_final.reshape(size1_final,1),y_final.reshape(size1_final,1),z_final.reshape(size1_final,1),a_final.reshape(size1_final,1),c_final.reshape(size1_final,1),X_final.reshape(size1_final,1),Y_final.reshape(size1_final,1),Z_final.reshape(size1_final,1),I_final.reshape(size1_final,1),J_final.reshape(size1_final,1),K_final.reshape(size1_final,1)),axis=1)
    dfout11_final = pd.DataFrame(conc11_final, columns=['x final','y final','z final','a final','c final','X final','Y final','Z final','I final','J final','K final'])
    
    #subplotter(0,final_df.shape[0],X_final,Y_final,Z_final,X_inv_final,Y_inv_final,Z_inv_final)   # using combined dataset
    
    return Y_final, final_df

In [ ]:
# Peak removal: by observing the data,
# it can be seen that the using just one of the components (in this case Y) all the outlers can be eliminated. 
count = 0
col = final_df.columns
while True:
    Y_final, final_df = remove_peaks(final_df,compensation_values_df)
    # Two types of spikes are typicalled observed (check visualizaion by runnung  vizualization.ipnyb) 
    # one in positive direction and othes in negetive direction, so for robostness both are considered. 
    # At the same time, it is not wise to remove the original expected trajectory of the tool path in begining and the end,
    # so especially for the positive direction they are intentionally not removed 
    print(-(np.mean(Y_final)-7*np.std(Y_final)), np.mean(Y_final), np.std(Y_final),(np.mean(Y_final)+7*np.std(Y_final)))
    peaks1, _ = find_peaks(-Y_final, height=(-(np.mean(Y_final)-7*np.std(Y_final)),None))
    #peaks2,_ = find_peaks(Y_final[0:-1000], height=(240,None))
    peaks2,_ = find_peaks(Y_final, height=((np.mean(Y_final)+7*np.std(Y_final)),None))
    peaks = list(peaks1) + list(peaks2)
    print(peaks)
    
    if(len(peaks)==0):
        break
        
    # if each peak on a plateu has to be removed its very expensive, so if two neighbous are consicutively classifies as peaks 
    # (given by count), it is assumed as plateu and 100 datapoints are removed.
    # This is a reasonable compromise in accuracy for significant speedup
        
    #elif(len(peaks)==1 and abs(Y_final[peaks[0]] - Y_final[peaks[0]+20] )<=0.3):
    elif(len(peaks)==1 and count>2 and peaks[-1]+100<final_df.shape[0]):
        arr = final_df.to_numpy()
        arr = np.delete(arr, np.arange(peaks[0],peaks[0]+100), 0)
        final_df = pd.DataFrame(arr,columns=col)
        count = 0
        
    elif(len(peaks)==1 and count>2 and (final_df.shape[0]-100<peaks[-1]<final_df.shape[0])):
        arr = final_df.to_numpy()
        arr = np.delete(arr, np.arange(peaks[0],peaks[0]+(final_df.shape[0]-peaks[-1])), 0)
        final_df = pd.DataFrame(arr,columns=col)
        count = 0
        
    else:
        arr = final_df.to_numpy()
        arr = np.delete(arr,peaks, 0)
        final_df = pd.DataFrame(arr,columns=col)
        count = count+1
        

In [ ]:
final_df.shape

### Visulaiztion of effect of removing peaks

In [ ]:
x_final = final_df['MachineX'].copy(deep=True)                 # using machine data from newly created combined dataset
y_final = final_df['MachineY'].copy(deep=True)
z_final = final_df['MachineZ'].copy(deep=True)
a_final = final_df['MachineA'].copy(deep=True)
c_final = final_df['MachineC'].copy(deep=True)

size1_final = x_final.shape[0]
print(size1_final)

# converting pandas series to numpy array
x_final = x_final.to_numpy()
y_final = y_final.to_numpy()
z_final = z_final.to_numpy()
a_final = a_final.to_numpy(dtype =  np.float64)
a_final = np.deg2rad(a_final)
c_final = c_final.to_numpy(dtype =  np.float64)
c_final = np.deg2rad(c_final)

tool_tip_X_final = final_df['Tool Tip Point X'].to_numpy()        # using newly created combined dataset
tool_tip_Y_final = final_df['Tool Tip Point Y'].to_numpy()
tool_tip_Z_final = final_df['Tool Tip Point Z'].to_numpy()
    
X_inv_final = tool_tip_X_final
Y_inv_final = tool_tip_Y_final
Z_inv_final = tool_tip_Z_final
    
#compensation_values
compensation_values = compensation_values_df.to_numpy()

# Within each cube we have ranges defined in x,y,z for the machine position

x_range = np.arange(-200,201,100)
y_range = np.arange(-300,301,150)
z_range = np.arange(-500,1,50)

obj3_final = compensation.Compensation(compensation_values,x_range,y_range,z_range)

# Caclculation of compensation error values based on machine positions obtained through inverse transformation
deltaX1_final, deltaY1_final, deltaZ1_final,deltaI1_final,deltaJ1_final,deltaK1_final = obj3_final.calculate(x_final,y_final,z_final)  
size3_final = x_final.shape[0]
conc3_final = np.concatenate((deltaX1_final.reshape(size3_final,1),deltaY1_final.reshape(size3_final,1),deltaZ1_final.reshape(size3_final,1),deltaI1_final.reshape(size3_final,1),deltaJ1_final.reshape(size3_final,1),deltaK1_final.reshape(size3_final,1)),axis=1)

# prininting the compensation error values
df_obj3_final = pd.DataFrame(conc3_final, columns=['deltaX1 final','deltaY1 final','deltaZ1 final','deltaI1 final','deltaJ1 final','deltaK1 final'])
df_obj3_final.head(5)  

x_compensated_final = x_final + deltaX1_final*10**-3    # since given compensation is to be converted from microns to mm (10**-6 x 10**3 = 10**-3)
y_compensated_final = y_final + deltaY1_final*10**-3
z_compensated_final = z_final + deltaZ1_final*10**-3

conc4_final = np.concatenate((x_compensated_final.reshape(size3_final,1),y_compensated_final.reshape(size3_final,1),z_compensated_final.reshape(size3_final,1)),axis=1)
df_obj4_final = pd.DataFrame(conc4_final, columns=['x_compensated final','y_compensated final','z_compensated final'])

obj_final = transformation.Transformation(size1_final,angle)
# Forward Transformation fuction:
# Input : Machine points in machine coordinate system
# Output: returns too tip points and orientation in workpiece coordinate system
tool_position_workpiece_CS_final, tool_orientation_workpiece_CS_final = obj_final.forward(x_compensated_final,y_compensated_final,z_compensated_final,a_final,c_final)

X_final = tool_position_workpiece_CS_final[0,0,:]
Y_final = tool_position_workpiece_CS_final[1,0,:]

Z_final = tool_position_workpiece_CS_final[2,0,:]

I_final = tool_orientation_workpiece_CS_final[0,0,:]
J_final = tool_orientation_workpiece_CS_final[1,0,:]
K_final = tool_orientation_workpiece_CS_final[2,0,:]

# Verification the correctness of code, i.e on applying reverse transformation on the forward transformation we should get same values
machine_points_xyz_final, machine_direction_ac_final = obj_final.backward(X_final,Y_final,Z_final,I_final,J_final,K_final)

x_out_final = machine_points_xyz_final[0,0,:]
y_out_final = machine_points_xyz_final[1,0,:]
z_out_final = machine_points_xyz_final[2,0,:]

a_out_final = machine_direction_ac_final[0,0,:]
c_out_final = machine_direction_ac_final[1,0,:]

conc1_final = np.concatenate((x_final.reshape(size1_final,1),x_out_final.reshape(size1_final,1),y_final.reshape(size1_final,1),y_out_final.reshape(size1_final,1),z_final.reshape(size1_final,1),z_out_final.reshape(size1_final,1),a_final.reshape(size1_final,1),a_out_final.reshape(size1_final,1),c_final.reshape(size1_final,1),c_out_final.reshape(size1_final,1)),axis=1)
df_obj1_final = pd.DataFrame(conc1_final, columns=['x final','x_out final','y final','y_out final','z final','z_out final','a final','a_out final','c final','c_out'])

conc11_final = np.concatenate((x_final.reshape(size1_final,1),y_final.reshape(size1_final,1),z_final.reshape(size1_final,1),a_final.reshape(size1_final,1),c_final.reshape(size1_final,1),X_final.reshape(size1_final,1),Y_final.reshape(size1_final,1),Z_final.reshape(size1_final,1),I_final.reshape(size1_final,1),J_final.reshape(size1_final,1),K_final.reshape(size1_final,1)),axis=1)
dfout11_final = pd.DataFrame(conc11_final, columns=['x final','y final','z final','a final','c final','X final','Y final','Z final','I final','J final','K final'])
    
subplotter(0,final_df.shape[0],X_final,Y_final,Z_final,X_inv_final,Y_inv_final,Z_inv_final)   # using combined dataset

In [ ]:
subplotter(0,1000,X_final,Y_final,Z_final,X_inv_final,Y_inv_final,Z_inv_final)

In [ ]:
def shift(final_df, compensation_values_df):
    x_final = final_df['MachineX']                 # using machine data from newly created combined dataset
    y_final = final_df['MachineY']
    z_final = final_df['MachineZ']
    a_final = final_df['MachineA']
    c_final = final_df['MachineC']

    size1_final = x_final.shape[0]

    # converting pandas series to numpy array
    x_final = x_final.to_numpy()
    y_final = y_final.to_numpy()
    z_final = z_final.to_numpy()
    a_final = a_final.to_numpy(dtype =  np.float64)
    a_final = np.deg2rad(a_final)
    c_final = c_final.to_numpy(dtype =  np.float64)
    c_final = np.deg2rad(c_final)

    tool_tip_X_final = final_df['Tool Tip Point X'].to_numpy()        # using newly created combined dataset
    tool_tip_Y_final = final_df['Tool Tip Point Y'].to_numpy()
    tool_tip_Z_final = final_df['Tool Tip Point Z'].to_numpy()

    X_inv_final = tool_tip_X_final
    Y_inv_final = tool_tip_Y_final
    Z_inv_final = tool_tip_Z_final

    #compensation_values
    compensation_values = compensation_values_df.to_numpy()

    # Within each cube we have ranges defined in x,y,z for the machine position

    x_range = np.arange(-200,201,100)
    y_range = np.arange(-300,301,150)
    z_range = np.arange(-500,1,50)

    obj3_final = compensation.Compensation(compensation_values,x_range,y_range,z_range)

    # Caclculation of compensation error values based on machine positions obtained through inverse transformation
    deltaX1_final, deltaY1_final, deltaZ1_final,deltaI1_final,deltaJ1_final,deltaK1_final = obj3_final.calculate(x_final,y_final,z_final)  

    x_compensated_final = x_final + deltaX1_final*10**-3    # since given compensation is to be converted from microns to mm (10**-6 x 10**3 = 10**-3)
    y_compensated_final = y_final + deltaY1_final*10**-3
    z_compensated_final = z_final + deltaZ1_final*10**-3

    obj_final = transformation.Transformation(size1_final,angle)
    # Forward Transformation fuction:
    # Input : Machine points in machine coordinate system
    # Output: returns too tip points and orientation in workpiece coordinate system
    tool_position_workpiece_CS_final, tool_orientation_workpiece_CS_final = obj_final.forward(x_compensated_final,y_compensated_final,z_compensated_final,a_final,c_final)

    X_final = tool_position_workpiece_CS_final[0,0,:]
    Y_final = tool_position_workpiece_CS_final[1,0,:]
    Z_final = tool_position_workpiece_CS_final[2,0,:]

    I_final = tool_orientation_workpiece_CS_final[0,0,:]
    J_final = tool_orientation_workpiece_CS_final[1,0,:]
    K_final = tool_orientation_workpiece_CS_final[2,0,:]

    #Shifting the data 
    div = 20
    length = Y_final.shape[0]
    chunck_len = int(length/div)
    
    for i in np.arange(0,div):
        if(i != div-1):
            diff = Y_final[i*chunck_len:(i+1)*chunck_len] - Y_inv_final[i*chunck_len:(i+1)*chunck_len]
            Y_final[i*chunck_len:(i+1)*chunck_len] = Y_final[i*chunck_len:(i+1)*chunck_len] - np.mean(diff)

        else:
            diff = Y_final[i*chunck_len:length] - Y_inv_final[i*chunck_len:length]
            Y_final[i*chunck_len:length] = Y_final[i*chunck_len:length] - np.mean(diff)

        
    return X_final,Y_final,Z_final



In [ ]:
X_mc, Y_mc, Z_mc = shift(final_df,compensation_values_df)

In [ ]:
final_df['Tool Tip Point Machine X'] = X_mc
final_df['Tool Tip Point Machine Y'] = Y_mc
final_df['Tool Tip Point Machine Z'] = Z_mc

In [ ]:
print(final_df.shape)
final_df.iloc[0:10]

In [ ]:
subplotter(19000,final_df.shape[0],final_df['Tool Tip Point Machine X'],final_df['Tool Tip Point Machine Y'],final_df['Tool Tip Point Machine Z'],final_df['Tool Tip Point X'],final_df['Tool Tip Point Y'],final_df['Tool Tip Point Z'])

In [ ]:
#tool point deviation

tool_point_deviation = np.sqrt(((final_df['Tool Tip Point Machine X'] - final_df['Tool Tip Point X'])**2 + (final_df['Tool Tip Point Machine Y'] - final_df['Tool Tip Point Y'])**2 + (final_df['Tool Tip Point Machine Z'] - final_df['Tool Tip Point Z'])**2).to_numpy(dtype=np.float64))
print(np.mean(tool_point_deviation))
plt.hist(tool_point_deviation, bins=50)
plt.show()

In [ ]:
final_df['Tool Point Deviation'] = tool_point_deviation

In [ ]:
final_df.iloc[10000:10020]

In [ ]:
plt.plot(a)

In [ ]:
plt.plot(c)

In [ ]:
plt.plot((final_df['tcp_error']).iloc[1000:3500])
plt.title('TCP Error plot')

In [ ]:
plt.figure(figsize=(10,15))
ax1 = plt.axes(projection='3d')
ax1.set_zlim3d([0,30])

ax1.set_xlabel('$X$ in mm', fontsize=20)
ax1.set_ylabel('$Y$ in mm', fontsize=20)
ax1.set_zlabel('$Z$ in mm', fontsize=20)

ax1.scatter3D(X_inv_final, Y_inv_final, Z_inv_final, color = 'red',label = 'Planning Data')
ax1.legend()

ax1.scatter3D(X_final, Y_final, Z_final, color = 'blue',label = 'Machine Data Avg')
ax1.legend()

In [ ]:
mean = final_df['tcp_error'].mean()
mean

In [ ]:
median = final_df['tcp_error'].median()
median

In [ ]:
minimum = final_df['tcp_error'].min()
minimum

In [ ]:
maximum = final_df['tcp_error'].max()  
maximum

In [ ]:
final_df = final_df.iloc[0:21500]

In [ ]:
# saving data 
final_df.to_excel(str(dir_final_save)+'finaldf_forward_with_compensation'+str(block)+'__'+str(angle)+'.xlsx') 